##### Copyright 2021 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2021 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/wav2vec2_saved_model_finetuning"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/hub/tutorials/wav2vec2_saved_model_finetuning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/hub/tutorials/wav2vec2_saved_model_finetuning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/hub/tutorials/wav2vec2_saved_model_finetuning.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
  <td>     <a href="https://tfhub.dev/vasudevgupta7/wav2vec2/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo do TF Hub</a>
</td>
</table>

# Ajustes finos de Wav2Vec2 com um head de modelagem de linguagem

Neste notebook, vamos carregar o modelo Wav2Vec2 pré-treinado a partir do [TF Hub](https://tfhub.dev) e fazer os ajustes finos com o [dataset LibriSpeech](https://huggingface.co/datasets/librispeech_asr), anexando um head de modelagem de linguagem (LM, na sigla em inglês) por cima do modelo pré-treinado. A tarefa subjacente é criar um modelo para **reconhecimento automático de fala**, isto é, dada uma fala, o modelo deve conseguir transcrevê-la em texto.

## Configuração

Antes de executar este notebook, confirme se você está usando o runtime de GPU (`Runtime` &gt; `Change runtime type` &gt; `GPU`). A célula abaixo instala o pacote [`gsoc-wav2vec2`](https://github.com/vasudevgupta7/gsoc-wav2vec2) e suas dependências.

In [ ]:
!pip3 install -q git+https://github.com/vasudevgupta7/gsoc-wav2vec2@main
!sudo apt-get install -y libsndfile1-dev
!pip3 install -q SoundFile

## Configuração do modelo usando o `TF Hub`

Vamos começar importando alguns módulos/bibliotecas.

In [ ]:
import os

import tensorflow as tf
import tensorflow_hub as hub
from wav2vec2 import Wav2Vec2Config

config = Wav2Vec2Config()

print("TF version:", tf.__version__)

Primeiro, vamos baixar o modelo do TF Hub e encapsular a assinatura do modelo com [`hub.KerasLayer`](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer) para podermos usar este modelo como uma camada do Keras. Felizmente, `hub.KerasLayer` consegue fazer isso com apenas uma linha.

**Observação:** ao carregar o modelo com `hub.KerasLayer`, ele se torna um pouco opaco, mas, às vezes, precisamos de controles mais finos do modelo, então podemos carregá-lo com `tf.keras.models.load_model(...)`.

In [ ]:
pretrained_layer = hub.KerasLayer("https://tfhub.dev/vasudevgupta7/wav2vec2/1", trainable=True)

Caso tenha interesse no script de exportação do modelo, confira [aqui](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/export2hub.py). O objeto `pretrained_layer` é a versão congelada de [`Wav2Vec2Model`](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/wav2vec2/modeling.py). Esses pesos pré-treinados foram convertidos a partir dos [pesos pré-treinados](https://huggingface.co/facebook/wav2vec2-base) de HuggingFace PyTorch usando [este script](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/convert_torch_to_tf.py).

Originalmente, Wav2Vec2 foi pré-treinado com uma estratégia de modelagem de linguagem mascarada, com o objetivo de identificar a representação de fala latente quantizada verdadeira para o timestep mascarado. Leia mais sobre o objetivo do treinamento no artigo [wav2vec 2.0 – Framework para aprendizado autossupervisionado de representações de fala](https://arxiv.org/abs/2006.11477).

Agora, vamos definir algumas constantes e hiperparâmetros, que serão úteis nas próximas células. `AUDIO_MAXLEN` é definido intencionalmente como `246000`, pois a assinatura do modelo aceita somente um tamanho de sequência estático igual a `246000`.

In [ ]:
AUDIO_MAXLEN = 246000
LABEL_MAXLEN = 256
BATCH_SIZE = 2

Na célula abaixo, vamos encapsular `pretrained_layer` e uma camada densa (head da LM) com a [API Functional do Keras](https://www.tensorflow.org/guide/keras/functional).

In [ ]:
inputs = tf.keras.Input(shape=(AUDIO_MAXLEN,))
hidden_states = pretrained_layer(inputs)
outputs = tf.keras.layers.Dense(config.vocab_size)(hidden_states)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

A camada densa (definida acima) tem uma dimensão de saída igual a `vocab_size`, pois queremos prever as probabilidades de cada token do vocabulário em cada timestep.

## Configuração do estado do treinamento

No TensorFlow, os pesos do modelo são criados somente quando `model.call` ou `model.build` é chamado pela primeira vez. Portanto, a célula abaixo cria os pesos do modelo. Além disso, vamos executar `model.summary()` para verificar o número total de parâmetros treináveis.

In [ ]:
model(tf.random.uniform(shape=(BATCH_SIZE, AUDIO_MAXLEN)))
model.summary()

Agora, precisamos definir a função de perda (`loss_fn`) e o otimizador para podermos treinar o modelo. A célula abaixo faz isso. Por questões de simplicidade, usaremos o otimizador `Adam`. `CTCLoss` é um tipo de perda comum usado para tarefas (como `ASR`) em que as subpartes da entrada não podem ser alinhadas facilmente às subpartes da saída. Saiba mais sobre CTC-loss nesta incrível [postagem de blog](https://distill.pub/2017/ctc/).

`CTCLoss` (do pacote [`gsoc-wav2vec2`](https://github.com/vasudevgupta7/gsoc-wav2vec2)) aceita três argumentos: `config`, `model_input_shape` e `division_factor`. Se `division_factor=1`, então a perda será simplesmente somada; portanto, passe `division_factor` para obter a média para o lote.

In [ ]:
from wav2vec2 import CTCLoss

LEARNING_RATE = 5e-5

loss_fn = CTCLoss(config, (BATCH_SIZE, AUDIO_MAXLEN), division_factor=BATCH_SIZE)
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

## Carregamento e pré-processamento dos dados

Agora, vamos baixar o dataset LibriSpeech no [site oficial](http://www.openslr.org/12) e configurá-lo.

In [ ]:
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz -P ./data/train/
!tar -xf ./data/train/dev-clean.tar.gz -C ./data/train/

**Observação:** estamos usando a configuração `dev-clean`, pois este notebook foi criado apenas para fins de demonstração, então precisamos de uma pequena quantidade de dados. Os dados de treinamento completos podem ser baixados no [site do LibriSpeech](http://www.openslr.org/12).

In [ ]:
ls ./data/train/

O dataset fica no diretório LibriSpeech. Vamos explorar os arquivos dentro dele.

In [ ]:
data_dir = "./data/train/LibriSpeech/dev-clean/2428/83705/"
all_files = os.listdir(data_dir)

flac_files = [f for f in all_files if f.endswith(".flac")]
txt_files = [f for f in all_files if f.endswith(".txt")]

print("Transcription files:", txt_files, "\nSound files:", flac_files)

Cada subdiretório tem diversos arquivos `.flac` e um arquivo `.txt`. O arquivo `.txt` contém as transcrições de texto para todas as amostras de fala (isto é, os arquivos `.flac`) presentes nesse subdiretório.

Podemos carregar os dados de texto da seguinte forma:

In [ ]:
def read_txt_file(f):
  with open(f, "r") as f:
    samples = f.read().split("\n")
    samples = {s.split()[0]: " ".join(s.split()[1:]) for s in samples if len(s.split()) > 2}
  return samples

De maneira similar, vamos definir uma função para carregar uma amostra de fala a partir de um arquivo `.flac`.

`REQUIRED_SAMPLE_RATE` é definido como `16000`, pois o Wav2Vec2 foi pré-treinado com frequência de `16K`, e recomenda-se fazer os ajustes finos sem uma grande mudança na distribuição dos dados devido à frequência.

In [ ]:
import soundfile as sf

REQUIRED_SAMPLE_RATE = 16000

def read_flac_file(file_path):
  with open(file_path, "rb") as f:
      audio, sample_rate = sf.read(f)
  if sample_rate != REQUIRED_SAMPLE_RATE:
      raise ValueError(
          f"sample rate (={sample_rate}) of your files must be {REQUIRED_SAMPLE_RATE}"
      )
  file_id = os.path.split(file_path)[-1][:-len(".flac")]
  return {file_id: audio}

Agora, vamos escolher algumas amostras aleatórias e tentar visualizá-las.

In [ ]:
from IPython.display import Audio
import random

file_id = random.choice([f[:-len(".flac")] for f in flac_files])
flac_file_path, txt_file_path = os.path.join(data_dir, f"{file_id}.flac"), os.path.join(data_dir, "2428-83705.trans.txt")

print("Text Transcription:", read_txt_file(txt_file_path)[file_id], "\nAudio:")
Audio(filename=flac_file_path)

Agora, vamos combinar todas as amostras de fala e texto, e também vamos definir a função (na próxima célula) para essa finalidade.

In [ ]:
def fetch_sound_text_mapping(data_dir):
  all_files = os.listdir(data_dir)

  flac_files = [os.path.join(data_dir, f) for f in all_files if f.endswith(".flac")]
  txt_files = [os.path.join(data_dir, f) for f in all_files if f.endswith(".txt")]

  txt_samples = {}
  for f in txt_files:
    txt_samples.update(read_txt_file(f))

  speech_samples = {}
  for f in flac_files:
    speech_samples.update(read_flac_file(f))

  assert len(txt_samples) == len(speech_samples)

  samples = [(speech_samples[file_id], txt_samples[file_id]) for file_id in speech_samples.keys() if len(speech_samples[file_id]) < AUDIO_MAXLEN]
  return samples

Chegou a hora de verificarmos algumas amostras...

In [ ]:
samples = fetch_sound_text_mapping(data_dir)
samples[:5]

Observação: estamos carregando esses dados na memória porque estamos trabalhando com um dataset pequeno neste notebook. Porém, para treinamento usando o dataset completo (cerca de 300 GB), você precisará carregar os dados de maneira lazy. Para mais informações, confira [este script](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/data_utils.py).

Vamos pré-processar os dados agora.

Primeiro, vamos definir o tokenizador e o processador usando o pacote `gsoc-wav2vec2`. Em seguida, vamos fazer um pré-processamento bem simples. O `processor` normalizará a fala bruta em relação ao eixo de frames, e o `tokenizer` converterá as saídas do modelo em string (usando o vocabulário definido) e cuidará da remoção de tokens especiais (dependendo da configuração do tokenizador).

In [ ]:
from wav2vec2 import Wav2Vec2Processor
tokenizer = Wav2Vec2Processor(is_tokenizer=True)
processor = Wav2Vec2Processor(is_tokenizer=False)

def preprocess_text(text):
  label = tokenizer(text)
  return tf.constant(label, dtype=tf.int32)

def preprocess_speech(audio):
  audio = tf.constant(audio, dtype=tf.float32)
  return processor(tf.transpose(audio))

Agora, vamos definir o gerador Python para chamar as funções de pré-processamento definidas na células acima.

In [ ]:
def inputs_generator():
  for speech, text in samples:
    yield preprocess_speech(speech), preprocess_text(text)

## Configuração do `tf.data.Dataset`

A célula abaixo configura o objeto `tf.data.Dataset` usando seu método `.from_generator(...)`. Usaremos o objeto `generator` definido na célula acima.

**Observação:** para treinamento distribuído (especialmente em TPUs), atualmente `.from_generator(...)` não funciona, e recomenda-se treinar usando dados armazenados no formato `.tfrecord` (idealmente, os TFRecords devem ser armazenados dentro de um bucket do GCS para que as TPUs sejam usadas ao máximo).

Confira mais detalhes de como converter dados LibriSpeech em TFRecords [neste script](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/make_tfrecords.py).

In [ ]:
output_signature = (
    tf.TensorSpec(shape=(None),  dtype=tf.float32),
    tf.TensorSpec(shape=(None), dtype=tf.int32),
)

dataset = tf.data.Dataset.from_generator(inputs_generator, output_signature=output_signature)

In [ ]:
BUFFER_SIZE = len(flac_files)
SEED = 42

dataset = dataset.shuffle(BUFFER_SIZE, seed=SEED)

Vamos passar o dataset para diversos lotes, então vamos preparar os lotes na célula abaixo. Todas as sequências em um lote devem ser preenchidas até um tamanho constante. Usaremos o método `.padded_batch(...)` para isso.

In [ ]:
dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=(AUDIO_MAXLEN, LABEL_MAXLEN), padding_values=(0.0, 0))

Os aceleradores (como GPUs/TPUs) são muito rápidos e, geralmente, o carregamento dos dados (e o pré-processamento) se torna o gargalo durante o treinamento, já que o carregamento ocorre em CPUs, o que pode aumentar bastante o tempo de treinamento, especialmente quando há muito pré-processamento online ou quando os dados são transmitidos online a partir de buckets do GCS. Para lidar com esses problemas, `tf.data.Dataset` conta com o método `.prefetch(...)`, que ajuda a preparar os próximos lotes paralelamente (em CPUs) enquanto o modelo está fazendo previsões (em GPUs/TPUs) no lote atual.

In [ ]:
dataset = dataset.prefetch(tf.data.AUTOTUNE)

Como este notebook foi criado para fins de demonstração, pegaremos somente os primeiros `num_train_batches` para fazer o treinamento. Porém, sugerimos que você faça o treinamento usando todo o dataset. De maneira similar, vamos avaliar somente `num_val_batches`.

In [ ]:
num_train_batches = 10
num_val_batches = 4

train_dataset = dataset.take(num_train_batches)
val_dataset = dataset.skip(num_train_batches).take(num_val_batches)

## Treinamento do modelo

Para o treinamento do modelo, vamos chamar o método `.fit(...)` diretamente após compilar o modelo com `.compile(...)`.

In [ ]:
model.compile(optimizer, loss=loss_fn)

A célula acima configura o estado do treinamento. Agora, podemos iniciar o treinamento com o método `.fit(...)`.

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)
history.history

Vamos salvar o modelo com o método `.save(...)` para podermos fazer a inferência posteriormente. Você também pode exportar esse SavedModel para o TF Hub de acordo com a [documentação do TF Hub](https://www.tensorflow.org/hub/publish).

In [ ]:
save_dir = "finetuned-wav2vec2"
model.save(save_dir, include_optimizer=False)

Observação: definimos `include_optimizer=False`, pois queremos usar este modelo somente para inferência.

## Avaliação

Agora, vamos computar a Taxa de Erro de Palavras para o dataset de validação.

A **Taxa de Erro de Palavras** (WER, na sigla em inglês) é uma métrica comum para mensurar o desempenho de um sistema de reconhecimento automático de fala. O WER é derivado da distância de Levenshtein no nível de palavra. Essa taxa pode ser computada da seguinte forma: WER = (S + D + I) / N = (S + D + I) / (S + D + C), em que S é o número de substituições, D é o número de exclusões, I é o número de inserções, C é o número de palavras corretas, e N é o número de palavras na referência (N=S+D+C). Esse valor indica a porcentagem de palavras que foram previstas incorretamente.

Para saber mais sobre WER, confira [este artigo](https://www.isca-speech.org/archive_v0/interspeech_2004/i04_2765.html).

Vamos usar a função `load_metric(...)` da biblioteca [HuggingFace datasets](https://huggingface.co/docs/datasets/). Primeiro, vamos instalar a biblioteca `datasets` usando `pip` e definir o objeto `metric`.

In [ ]:
!pip3 install -q datasets

from datasets import load_metric
metric = load_metric("wer")

In [ ]:
@tf.function(jit_compile=True)
def eval_fwd(batch):
  logits = model(batch, training=False)
  return tf.argmax(logits, axis=-1)

Agora está na hora de executar a avaliação para os dados de validação.

In [ ]:
from tqdm.auto import tqdm

for speech, labels in tqdm(val_dataset, total=num_val_batches):
    predictions  = eval_fwd(speech)
    predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
    references = [tokenizer.decode(label, group_tokens=False) for label in labels.numpy().tolist()]
    metric.add_batch(references=references, predictions=predictions)

Usamos o método `tokenizer.decode(...)` para decodificar as previsões e os rótulos de volta para texto e adicionamos à métrica para a computação de `WER` posteriormente.

Agora, vamos calcular o valor da métrica na célula abaixo:

In [ ]:
metric.compute()

**Observação:** aqui, o valor da métrica não faz nenhum sentido, já que o modelo é treinado com poucos dados, e tarefas de reconhecimento automático de fala costumam exigir muitos dados para aprender um mapeamento de fala para texto. Para conseguir bons resultados, provavelmente você precisará treinar com muitos dados. Este notebook fornece um template para fazer os ajustes finos de um modelo de fala pré-treinado.

## Inferência

Agora que estamos satisfeitos com o processo de treinamento e salvamos o modelo em `save_dir`, veremos como ele pode ser usado para inferência.

Primeiro, vamos carregar o modelo usando `tf.keras.models.load_model(...)`.

In [ ]:
finetuned_model = tf.keras.models.load_model(save_dir)

Vamos baixar algumas amostras de fala para fazer a inferência. Você pode substituir a amostra abaixo pela sua amostra de fala.

In [ ]:
!wget https://github.com/vasudevgupta7/gsoc-wav2vec2/raw/main/data/SA2.wav

Agora, vamos ler a amostra de fala usando `soundfile.read(...)` e preenchê-la até `AUDIO_MAXLEN` para atender à assinatura do modelo. Em seguida, vamos normalizar essa amostra de fala usando a instância de `Wav2Vec2Processor` e vamos alimentá-la ao modelo.

In [ ]:
import numpy as np

speech, _ = sf.read("SA2.wav")
speech = np.pad(speech, (0, AUDIO_MAXLEN - len(speech)))
speech = tf.expand_dims(processor(tf.constant(speech)), 0)

outputs = finetuned_model(speech)
outputs

Vamos decodificar os números de volta na sequência de texto usando a instância de `Wav2Vec2tokenizer` definida acima.

In [ ]:
predictions = tf.argmax(outputs, axis=-1)
predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
predictions

Essa previsão é bem aleatória, pois o modelo nunca foi treinado com muitos dados neste notebook (não é objetivo deste notebook fazer um treinamento completo). Você conseguirá boas previsões se treinar este modelo usando o dataset LibriSpeech completo.

Chegamos ao fim deste notebook, mas não é o fim do aprendizado de tarefas relacionadas a fala no TensorFlow. Este [repositório](https://github.com/tulasiram58827/TTS_TFLite) contém alguns tutoriais incríveis. Caso você encontre algum bug neste notebook, pedimos que crie um issue [aqui](https://github.com/vasudevgupta7/gsoc-wav2vec2/issues).